In [1]:
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd # working with tabular data
import pickle # for machine learning models
import seaborn as sns # visualization
import matplotlib.pyplot as plt # visualization

from sklearn.feature_extraction import DictVectorizer # Machine Learning
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn.linear_model import Lasso # Regularization
from sklearn.linear_model import Ridge # Regularization

from sklearn.metrics import mean_squared_error # Loss Function

In [3]:
import mlflow

# to hook up with MLFlow UI
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mlops_nyc_taxi") # choose a name for your experiment

/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2025/03/16 19:39:42 INFO mlflow.tracking.fluent: Experiment with name 'mlops_nyc_taxi' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/mlruns/1', creation_time=1742128782056, experiment_id='1', last_update_time=1742128782056, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

--2025-03-16 19:41:23--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.197, 3.164.82.160, 3.164.82.112, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet’

yellow_tripdata_202 100%[===================>]  36.37M  10.5MB/s    in 3.5s    

2025-03-16 19:41:27 (10.5 MB/s) - ‘yellow_tripdata_2022-01.parquet’ saved [38139949/38139949]

--2025-03-16 19:41:27--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Reusing existing connection to d37ci6vzurychx.cloudfront.net:443.
HTTP request sent, awaiting response... 200 OK
Length: 45616512 (44M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-02.parquet’

yellow_tripdata

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('yellow_tripdata_2022-02.parquet')

In [7]:
len(df_train), len(df_val)

(2421440, 2918187)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

30.585991358919788

In [16]:
with open('./lin_reg.bin', 'wb') as f_out: # wb means write binary mlops-zoomcamp/week_1/models
    try:
        # Pickle both the dictionary vectorizer and the linear regression model
        pickle.dump((dv, lr), f_out)
        print("Model successfully pickled.")
    except Exception as e:
        print("Error occurred while pickling the model:", str(e))

Model successfully pickled.


In [17]:
!pip install xgboost
!pip install hyperopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00a 0:00:01
Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 12.5 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [18]:
# import required modules
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials # some methods to optimize hyperparameters
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
# Objective function - set the parameters for this specific run
def objective(params): 
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params) 
        booster = xgb.train(
            params=params,
            dtrain=train, # model trained on training set
            num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
            evals=[(valid, 'validation')], # model evaluated on validation set
            early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
# Set the range of the hyperparameter optimization search
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), # tree depth 4 to 100. Returns float, so convert to integer
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # range exp(-3), exp(0) which is (0.049787, 1.0)
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1), # range exp(-5), exp(-1) which is (0.006738, 0.367879)
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1), # range exp(-6), exp(-1) which is (0.002479, 0.367879)
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3), # range exp(-1), exp(3) which is (0.367879, 20.085537)
    'objective': 'reg:linear',
    'seed': 42
}


best_result = fmin( # imported above
    fn=objective,
    space=search_space, # as defined above
    algo=tpe.suggest, # tpe is the algorithm used for optimization
    max_evals=3, #restricted, time constraints
    trials=Trials()
)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:52:13] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.02699                          
[1]	validation-rmse:8.43783                          
[2]	validation-rmse:7.92976                          
 33%|███▎      | 1/3 [00:12<00:24, 12.16s/trial, best loss: 62.88115283167555]

/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:52:25] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.64848                                                   
[1]	validation-rmse:6.47024                                                   
[2]	validation-rmse:5.82476                                                   
 67%|██████▋   | 2/3 [00:25<00:12, 12.65s/trial, best loss: 33.927832593170606]

/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:52:38] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.94649                                                    
[1]	validation-rmse:8.30012                                                    
[2]	validation-rmse:7.75431                                                    
100%|██████████| 3/3 [00:37<00:00, 12.40s/trial, best loss: 33.927832593170606]


In [24]:
mlflow.xgboost.autolog(disable=True) # MLflow will not store parameters automatically - these will have to be requested

In [25]:
#best params

params = {
    'learning_rate': 0.33500504398153885,
    'max_depth': 74,
    'min_child_weight': 3.4593376740383635,
    'objective': 'reg:linear',
    'reg_alpha': 0.02439429006088758,
    'reg_lambda': 0.0029162481576915956,
    'seed': 42   
}

In [28]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.4434065752589766,
        'max_depth': 81,
        'min_child_weight': 10.423237853746643,
        'objective': 'reg:linear',
        'reg_alpha': 0.2630756846813668,
        'reg_lambda': 0.1220536223877784,
        'seed': 42    
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=params,
        dtrain=train, # model trained on training set
        num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
        evals=[(valid, 'validation')], # model evaluated on validation set
        early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("preprocessor.b", "wb") as f_out: # save pre-processing as a model
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("preprocessor.b", artifact_path="preprocessor") # we can isolate the pre-processing from raw data
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow") 

/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:11:20] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:7.64848
[1]	validation-rmse:6.47024
[2]	validation-rmse:5.82476


/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:11:35] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/03/16 22:11:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [29]:
logged_model = 'mlruns/1/b0cbc283e72241b99cf27d0352dc1088/artifacts/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [30]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: b0cbc283e72241b99cf27d0352dc1088

In [31]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [32]:
xgboost_model

In [33]:
y_pred = xgboost_model.predict(valid)

In [34]:
# check the first 10
y_pred[:10]

array([14.190828, 18.68855 , 26.193169, 23.354773, 26.193169, 13.043591,
       21.33266 ,  6.970524, 15.222571, 15.575311], dtype=float32)

Model Registry